<a href="https://colab.research.google.com/github/ashrafulwork/TrafficVisionBD/blob/main/TrafficVisionBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"mdashrafulhossain","key":"2e9580bff8453309162e28b9d2e05ca9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hasibzunair/rsud20k-bangladesh-road-scene-understanding # Replace 'datasetname' with your dataset's identifier


Dataset URL: https://www.kaggle.com/datasets/hasibzunair/rsud20k-bangladesh-road-scene-understanding
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
rsud20k-bangladesh-road-scene-understanding.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
import os

# Assuming the dataset is in a single zip file, e.g., 'datasetname.zip'
with zipfile.ZipFile("rsud20k-bangladesh-road-scene-understanding.zip", 'r') as zip_ref:
    zip_ref.extractall("path/to/extract")  # Replace with your preferred directory


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Set up ImageDataGenerator to load images with labels
data_dir = '/content/path/to/extract/rsud20k'  # Replace with the path where images are extracted

# Load images directly from the directory
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
data = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),  # Resize images to 128x128 (adjust as needed)
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' if you have more than 2 classes
)

# Display a batch of images
def show_images(images, labels):
    plt.figure(figsize=(10, 10))
    for i in range(9):  # Display 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(f"Label: {int(labels[i])}")
        plt.axis("off")
    plt.show()

# Get a batch of images and labels
images, labels = next(data)
show_images(images, labels)


Found 20334 images belonging to 2 classes.


In [ ]:
!pip install yolov5
from yolov5 import YOLOv5
yolo = YOLOv5("yolov5s.pt")  # load a pre-trained YOLO model


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
results = model(source = "loo.mp4", show =True, conf =0.4, save = True)


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/406) /content/loo.mp4: 384x640 1 person, 1 motorcycle, 1 bus, 4 trucks, 254.7ms
video 1/1 (frame 2/406) /content/loo.mp4: 384x640 1 person, 1 motorcycle, 4 trucks, 239.0ms
video 1/1 (frame 3/406) /content/loo.mp4: 384x640 2 persons, 1 car, 1 motorcycle, 3 trucks, 238.9ms
video 1/1 (frame 4/406) /content/loo.mp4: 384x640 1 person, 1 motorcycle, 2 trucks, 256.5ms

In [ ]:
import pandas as pd

# Read the labels from the label.txt file
labels_file_path = '/content/path/to/extract/rsud20k/labels'  # Replace with the actual path to your label.txt
labels_df = pd.read_csv(labels_file_path, sep=' ', header=None, names=['filename', 'label'])

# Display the first few rows of the dataframe
print(labels_df.head())


IsADirectoryError: [Errno 21] Is a directory: '/content/path/to/extract/rsud20k/labels'

In [ ]:
import os

# Path to the train directory
train_dir = '/content/path/to/extract/rsud20k/labels/train'  # Adjust this path accordingly

# List all text files in the train directory
label_files = [f for f in os.listdir(train_dir) if f.endswith('.txt')]
print(label_files)  # This will show all the label text files


['train10371.txt', 'train3924.txt', 'train3614.txt', 'train1358.txt', 'train4044.txt', 'train1866.txt', 'train2296.txt', 'train15969.txt', 'train11949.txt', 'train14706.txt', 'train11951.txt', 'train637.txt', 'train3238.txt', 'train5921.txt', 'train10746.txt', 'train108.txt', 'train70.txt', 'train7911.txt', 'train11653.txt', 'train3672.txt', 'train4252.txt', 'train4876.txt', 'train11608.txt', 'train2266.txt', 'train9553.txt', 'train4289.txt', 'train17266.txt', 'train16303.txt', 'train12092.txt', 'train9025.txt', 'train3009.txt', 'train4960.txt', 'train14369.txt', 'train2538.txt', 'train4899.txt', 'train8091.txt', 'train17927.txt', 'train732.txt', 'train12415.txt', 'train17627.txt', 'train15369.txt', 'train178.txt', 'train10531.txt', 'train866.txt', 'train2672.txt', 'train13340.txt', 'train8906.txt', 'train467.txt', 'train1993.txt', 'train14042.txt', 'train4138.txt', 'train10352.txt', 'train736.txt', 'train5465.txt', 'train13565.txt', 'train3326.txt', 'train14514.txt', 'train12912.txt',

In [ ]:
import pandas as pd

# Initialize an empty DataFrame to hold all labels
all_labels = pd.DataFrame(columns=['filename', 'label'])

# Read each label file and append to the DataFrame
for label_file in label_files:
    label_file_path = os.path.join(train_dir, label_file)  # Full path to the label file
    # Read the labels from the current file
    labels_df = pd.read_csv(label_file_path, sep=' ', header=None, names=['filename', 'label'])
    all_labels = pd.concat([all_labels, labels_df], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(all_labels.head())


<ipython-input-26-b28b93a89d8f>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_labels = pd.concat([all_labels, labels_df], ignore_index=True)


   filename     label
0  0.067187  0.341667
1  0.032813  0.075000
2  0.084375  0.275926
3  0.159375  0.430556
4  0.160938  0.340741


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Directory where the train images are stored
images_dir = '/content/path/to/extract/rsud20k/images/train'  # Adjust this path accordingly

# Function to display images with their labels
def show_annotated_images(labels_df, images_dir, num_images=9):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        # Get the filename and label
        row = labels_df.iloc[i]
        img_path = os.path.join(images_dir, row['filename'])
        label = row['label']

        # Load and display the image
        img = Image.open(img_path)
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.title(f"Label: {label}")
        plt.axis("off")
    plt.show()

# Show annotated images
show_annotated_images(all_labels, images_dir)


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'float64'

<Figure size 1000x1000 with 0 Axes>